# Pinhole Camera Model Practice Questions

## Question 1: Perspective Projection

A 3D point in the camera coordinate system is given by `p_c = [10, 5, 20]`. 
The camera has a focal length `f` of 50mm. 
Calculate the position of the projection of this point on the image plane in meters.

In [1]:
f = 0.05
p_c = [10, 5, 20]
p_2d = [f*(p_c[0]/p_c[2]), f*(p_c[1]/p_c[2])]
print(p_2d)

[0.025, 0.0125]


---

## Question 2: Conversion to Pixels

Given the projected point from Question 1, convert its coordinates from meters to pixels. 
Assume the following intrinsic parameters: 
- `s_x` = 800 pixels/meter
- `s_y` = 800 pixels/meter
- `o_x` = 320 pixels
- `o_y` = 240 pixels
What are the pixel coordinates `(u^I, v^I)`?

In [2]:
s_x = 800
s_y = 800
o_x = 320
o_y = 240
u_cm = p_2d[0]
v_cm = p_2d[1]
p_2d_px = [s_x * u_cm + o_x, s_y * v_cm + o_y]
print(p_2d_px)

[340.0, 250.0]


---

## Question 3: Intrinsic Matrix

Construct the intrinsic calibration matrix `K` using the parameters from Question 2, assuming no skew (`s_θ = 0`).

In [3]:
import numpy as np
matrix = np.array([[s_x * f, 0, o_x],
                   [0, s_y * f, o_y],
                   [0, 0 ,1]])
print(matrix)

[[ 40.   0. 320.]
 [  0.  40. 240.]
 [  0.   0.   1.]]


---

## Question 4: Point Projection with Known Pose

A point in the world frame is given by `p_w = [2, 3, 50]`. The camera's pose `T_w^c` is given by an identity rotation matrix and a translation vector `t_w^c = [0, 0, 10]`. [cite_start]Using the intrinsic matrix `K` from Question 3, calculate the pixel coordinates of the projected point. [cite: 14, 16, 17]

In [4]:
R = np.eye(3)
t = np.array([[0], [0], [10]])  
extrinsic_matrix = np.hstack([R,t])

p_w = np.array([[2], [3], [50], [1]])

intrinsic_and_extrinsic = matrix @ extrinsic_matrix
# print(intrinsic_and_extrinsic)

leftside = matrix @ extrinsic_matrix @ p_w
print(leftside)
pixel_coords = leftside[:2] / leftside[2]

print(pixel_coords.flatten())

[[19280.]
 [14520.]
 [   60.]]
[321.33333333 242.        ]


---

## Question 5: Radial Distortion

A point is projected on the image plane at `(u_distort^I, v_distort^I) = (400, 350)` including radial distortion. Given distortion coefficients `a1 = 0.0001` and `a2 = 0.0000002` and the principal point `(o_x, o_y) = (320, 240)`, calculate the corrected pixel coordinates `(u^I, v^I)`.

In [5]:
u_distort_I = 400
v_distort_I = 350
a1 = 0.0001
a2 = 0.0000002
o_x = 320
o_y = 240
r_sq = (u_distort_I - o_x)**2 + (v_distort_I - o_y)**2

u_I = (1+a1*r_sq + a2*r_sq**2) * (u_distort_I - o_x) + o_x
v_I = (1+a1*r_sq + a2*r_sq**2) * (v_distort_I - o_y) + o_y
print(u_I, v_I)

6024.0 8083.0
